## Defining the Problem

this notebook has been created on kaggle were the dataset is located so you have to import it to kaggle

# analysis

### import library

In [ ]:
import pandas as pd
import numpy as np
import os

from tqdm.notebook import tqdm
from PIL import Image

from ast import literal_eval




df = pd.read_csv('../input/nft-art-dataset/dataset/dataset.csv')
df.head()

creating new dataset with images only

In [ ]:
df_img = df[df['type'] == 'PHOTO']
df_img.head()

# Data Preprocessing

We have image and text data.

* Image data: resize
* Text data: tokenization and converting to integer IDs


In [ ]:
# preprocess image 500ta
import os
from PIL import Image

def load_image(file):

    try:
        image = Image.open('../input/nft-art-dataset' + 
            file[1:]
        ).convert('LA').resize((64, 64))
        arr = np.array(image)
    except:
        arr = np.zeros((64, 64, 2))
    return arr

# loading images:
x_train_image = np.array([load_image(i) for i in tqdm(df_img.path)])

# loading overview: (force convert some of the non-string cell to string)
x_train_text = df_img.art_series.astype('str')

# get price 
y_train_price = df_img.price

# get nsfw

y_train_nsfw = df_img.nsfw.astype(int)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_train_image[0, :, :, 0])

# Text Preprocessing

In [ ]:
# preprocess text data

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pprint import pprint

vocab_size = 40000
max_len = 100


# build vocabulary from training set
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train_text)


def _preprocess(list_of_text):
    return pad_sequences(
        tokenizer.texts_to_sequences(list_of_text),
        maxlen=max_len,
        padding='post',
    )
    

# padding is done inside: 
x_train_text_id = _preprocess(x_train_text)

print(x_train_text_id.shape)

In [ ]:
# we can use the tokenizer to convert IDs to words.
pprint(tokenizer.sequences_to_texts(x_train_text_id[:10]))

# Building model

# Trial 1

# image regression

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import GlobalAveragePooling2D, Rescaling
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import GRU


in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 2))

# text part
# simple average of embedding. you can change it to anything else as needed
embedded = keras.layers.Embedding(tokenizer.num_words, 100)(in_text)

# adding GRU layer after embedding layer and
attention = MultiHeadAttention(num_heads=2, key_dim=2)(embedded, embedded)
bi_lstm = Bidirectional(GRU(50, recurrent_regularizer='l2'))(attention)


scaling = Rescaling(1./127.5, offset=-1)(in_image) # rescaling
pl = GlobalAveragePooling2D()(scaling) # global average pooling, pretty much the same as flattening
do = Dropout(0.2)(pl) # dropout at a rate of 20%
fl = Flatten()(do)

# image part 
# simple conv2d. you can change it to anything else as needed
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)


# fusion - combinig both
fused = tf.concat([bi_lstm, flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task
p_price = Dense(1, activation='linear', name='price')(fused)


# define model input/output using keys.
bi_lstm_model = keras.Model(
    inputs={
        'ِart_series': in_text,
        'image': in_image
    },
    outputs={
        'price': p_price,

    }
)


# compile model with optimizer, loss values for each task, loss 
# weights for each task.
bi_lstm_model.compile(
    optimizer=Adam(),
    loss={
        'price': 'mean_squared_error',
    },
    loss_weights={
        'price': 0.5,       
    },
    metrics={
        'price': ['mse'],
    },
)


bi_lstm_model.summary()

In [ ]:
history = bi_lstm_model.fit(
    x={
        'image': x_train_image
    },
    y={
        'price': y_train_price_log
    },
    epochs=20,
    batch_size=16,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, )
    ],
    verbose=1
)

# images only predict price

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import collections
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention
from tensorflow.keras.layers import GlobalAveragePooling2D, Rescaling
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import GRU


in_image = keras.Input(batch_shape=(None, 64, 64, 2))



scaling = Rescaling(1./127.5, offset=-1)(in_image) # rescaling
pl = GlobalAveragePooling2D()(scaling) # global average pooling, pretty much the same as flattening
do = Dropout(0.2)(pl) # dropout at a rate of 20%
fl = Flatten()(do)

# image part 
# simple conv2d. you can change it to anything else as needed
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)



# multi-task learning (each is a multi-class classification)
# one dense layer for each task
p_price = Dense(1, activation='linear', name='price')(flattened)


# define model input/output using keys.
bi_lstm_model = keras.Model(
    inputs={
        'image': in_image
    },
    outputs={
        'price': p_price,

    }
)


# compile model with optimizer, loss values for each task, loss 
# weights for each task.
bi_lstm_model.compile(
    optimizer=Adam(),
    loss={
        'price': 'mean_squared_error',
    },
    loss_weights={
        'price': 0.5,       
    },
    metrics={
        'price': ['mse'],
    },
)


bi_lstm_model.summary()

In [ ]:
history = bi_lstm_model.fit(
    x={
        'ِart_series': x_train_text_id,
        'image': x_train_image
    },
    y={
        'price': y_train_price_log
    },
    epochs=20,
    batch_size=16,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, )
    ],
    verbose=1
)

# nsfw  (text)

In [ ]:
!pip install tensorflow_text

In [ ]:
!python -m pip install tensorflow_text

In [ ]:
pip install -q -U "tensorflow-text==2.8.*"

In [ ]:
!pip install /kaggle/input/tensorflow-text211/tensorflow_text-2.1.1-cp36-cp36m-manylinux1_x86_64.whl

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
df_balanced = pd.concat([df_not_nsfw_downsampled, df_nsfw])
df_balanced.shape

In [ ]:
df_balanced

In [ ]:
df_balanced['nsfw'].value_counts()

In [ ]:
df_balanced['nsfw']=df_balanced['nsfw'].apply(lambda x: 1 if x==True else 0)
df_balanced.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['art_series'],df_balanced['nsfw'], stratify=df_balanced['nsfw'])

import BERT model and get embeding vectors for few sample statements

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']


In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=30)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 